In [1]:
from tensorflow import keras

import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

In [3]:
dataset,info = tfds.load("tf_flowers",as_supervised=True,with_info=True)
dataset_size = info.splits["train"].num_examples#3670
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes#5

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


拆分训练集

In [4]:
# data = tfds.load('tf_flowers', split=['train[:10%]', 'train[10%:25%]', 'train[25%:]'])
# train_data, valid_data, test_data = data[0], data[1], data[2]
train_data = tfds.load('tf_flowers', split='train[:10%]', as_supervised=True)
valid_data = tfds.load('tf_flowers', split='train[10%:25%]', as_supervised=True)
test_data = tfds.load('tf_flowers', split='train[25%:]', as_supervised=True)



In [5]:
def preprocess(image,label):
  resized_image = tf.image.resize(image,[224,224])
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image,label

## 数据增强

In [28]:
def preprocess(image, label):
    # Apply image augmentation techniques
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.0, upper=1.0)

    # Normalize pixel values to [-1, 1]
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0

    return image, label

In [6]:
batch_size = 32
train_set = train_data.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_data.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_data.map(preprocess).batch(batch_size).prefetch(1)

加载一个在imagenet上预训练的Xception模型，设置include_top=False排除网络的顶部：这排除了全局平均池化层和密集输出层。

In [7]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)


83683744/83683744 [==============================] - 0s 0us/step


In [8]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes,activation="softmax")(avg)
model=keras.Model(inputs=base_model.input,outputs=output)

通常情况下，在使用预训练模型进行迁移学习时，冻结(或固定)预训练模型的权重是一个好主意，原因如下：

减少训练时间：预训练模型通常训练了大量的图像，因此预训练层的权重包含了许多已经学习到的图像特征，这些特征可以用来处理新的数据。将这些权重冻结，可以避免在训练初期过多地调整它们，从而加快模型的训练速度。
避免过拟合：预训练层的权重已经被训练成能够识别一类图像的特征。如果在训练初期就开始调整这些权重，可能会导致模型过拟合，因为模型会很快地开始依赖已有的数据。如果您的数据集较小，则更容易发生过拟合，因此应该尽可能减少对预训练层的调整。
更好地利用预训练模型的知识：预训练模型的权重已经被调整到了一个较好的状态，可以识别图像中的一些常见特征。因此，如果在训练期间冻结这些层，模型就能够更好地利用这些知识，从而提高模型的性能和泛化能力。
尽管冻结预训练层的权重的好处非常明显，但在某些情况下调整预训练层的权重也可能是合适的。例如，在您的新数据集与预训练数据集之间存在很大的不同之处时，你可能需要对预训练模型进行细调，从而适应您的数据。

In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [10]:
optimizer = tf.keras.optimizers.legacy.SGD(lr=0.2,momentum=0.9,decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
history = model.fit(train_set,epochs=5,validation_data=valid_set)

Epoch 1/5
12/12 [==============================] - 19s 502ms/step - loss: 2.4987 - accuracy: 0.5722 - val_loss: 1.2068 - val_accuracy: 0.7840
Epoch 2/5
12/12 [==============================] - 4s 306ms/step - loss: 0.7800 - accuracy: 0.8883 - val_loss: 1.4117 - val_accuracy: 0.8058
Epoch 3/5
12/12 [==============================] - 4s 350ms/step - loss: 0.3665 - accuracy: 0.9101 - val_loss: 1.4425 - val_accuracy: 0.8040
Epoch 4/5
12/12 [==============================] - 4s 307ms/step - loss: 0.3070 - accuracy: 0.9292 - val_loss: 1.4595 - val_accuracy: 0.8167
Epoch 5/5
12/12 [==============================] - 4s 309ms/step - loss: 0.1557 - accuracy: 0.9673 - val_loss: 1.5714 - val_accuracy: 0.8149


## 解冻所有层

In [13]:
for layer in base_model.layers:
  layer.trainable=True

optimizer = tf.keras.optimizers.legacy.SGD(lr=0.001,momentum=0.9,decay=0.001)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,epochs=5,validation_data=valid_set)

Epoch 1/5
12/12 [==============================] - 15s 768ms/step - loss: 0.0202 - accuracy: 0.9918 - val_loss: 1.6163 - val_accuracy: 0.7895
Epoch 2/5
12/12 [==============================] - 8s 672ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.4279 - val_accuracy: 0.7967
Epoch 3/5
12/12 [==============================] - 8s 678ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.2892 - val_accuracy: 0.8094
Epoch 4/5
12/12 [==============================] - 8s 662ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 1.1852 - val_accuracy: 0.8221
Epoch 5/5
12/12 [==============================] - 8s 694ms/step - loss: 0.0066 - accuracy: 0.9973 - val_loss: 1.1016 - val_accuracy: 0.8276
